In [2]:
import pandas as pd
import numpy as np
#dev
import importlib as imp
import functions
from functions import *
import functions_data_prep
from functions_data_prep import *


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
BASE_PATH = "datasets/DAIC-WOZ/"
files = os.listdir(BASE_PATH+"/cuts")

### Read in pandas

In [4]:
df = pd.read_csv(BASE_PATH + "Patient_Classes.csv")

In [5]:
df

,Participant_ID,PHQ8_Binary,PHQ8_Score,PHQ8_multiclass,dataset
0,303,0,0,0,train
1,304,0,6,1,train
2,305,0,7,1,train
3,310,0,4,0,train
4,312,0,2,0,train
...,...,...,...,...,...
137,483,1,15,3,dev
138,484,0,9,1,dev
139,489,0,3,0,dev
140,490,0,2,0,dev


### SAve AVG stuff

In [6]:
imp.reload(functions_data_prep)
from functions_data_prep import *

BASE_PATH_CUTS = "datasets/DAIC-WOZ/cuts/"
PATIENT = 419

MODE = "Binary" # or Multiclass

# Path to the .npy file
feature_path = BASE_PATH_CUTS+f"best_paper_features_only_mfcc_{str(PATIENT)}.npy"

In [ ]:
imp.reload(functions_data_prep)
from functions_data_prep import *

# Usage

organized_features = load_and_organize_features(feature_path)

# Now you can access features for each chunk like this:
# first_chunk_features = organized_features[0]
#mfcc_means = first_chunk_features['mfcc_mean']
#pitch = first_chunk_features['pitch']

In [12]:
def create_feature_summary_df(organized_features):
    """
    Create a DataFrame with min, max, and avg values for all features across chunks.
    """
    # First collect all values for each feature
    feature_values = {
        'mfcc_mean': np.array([chunk['mfcc_mean'] for chunk in organized_features]),
        'mfcc_std': np.array([chunk['mfcc_std'] for chunk in organized_features]),
        'spectral_centroid': np.array([chunk['spectral_centroid_mean'] for chunk in organized_features]),
        'spectral_bandwidth': np.array([chunk['spectral_bandwidth_mean'] for chunk in organized_features]),
        'spectral_rolloff': np.array([chunk['spectral_rolloff_mean'] for chunk in organized_features]),
        'zero_crossing_rate': np.array([chunk['zero_crossing_rate_mean'] for chunk in organized_features]),
        'pitch': np.array([chunk['pitch'] for chunk in organized_features]),
        'energy': np.array([chunk['energy'] for chunk in organized_features]),
        'speech_rate': np.array([chunk['speech_rate'] for chunk in organized_features])
    }
    
    # Create a dictionary to store all features
    all_features = {}
    
    # Handle MFCC coefficients (40 of them)
    for i in range(40):
        # For MFCC means
        all_features[f'mfcc_mean_{i}_min'] = np.min(feature_values['mfcc_mean'][:, i])
        all_features[f'mfcc_mean_{i}_max'] = np.max(feature_values['mfcc_mean'][:, i])
        all_features[f'mfcc_mean_{i}_avg'] = np.mean(feature_values['mfcc_mean'][:, i])
        
        # For MFCC standard deviations
        all_features[f'mfcc_std_{i}_min'] = np.min(feature_values['mfcc_std'][:, i])
        all_features[f'mfcc_std_{i}_max'] = np.max(feature_values['mfcc_std'][:, i])
        all_features[f'mfcc_std_{i}_avg'] = np.mean(feature_values['mfcc_std'][:, i])
    
    # Handle other features
    for feature in ['spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 
                   'zero_crossing_rate', 'pitch', 'energy', 'speech_rate']:
        all_features[f'{feature}_min'] = np.min(feature_values[feature])
        all_features[f'{feature}_max'] = np.max(feature_values[feature])
        all_features[f'{feature}_avg'] = np.mean(feature_values[feature])
    
    # Create DataFrame with a single row
    df = pd.DataFrame([all_features])
    
    return df

In [20]:
for _, row in df.iterrows():
    patient_id =  row['Participant_ID']
    try:
        feature_path = BASE_PATH_CUTS+f"best_paper_features_only_mfcc_{str(patient_id)}.npy"
        organized_features = load_and_organize_features(feature_path)

        create_feature_summary_df(organized_features).to_csv(f"{BASE_PATH_CUTS}_AVG_MIN_MAX_{patient_id}.csv")
    except:
        print(f"Error with patient {patient_id}")
        continue

Error with patient 491
Error with patient 489
Error with patient 490
Error with patient 492


In [30]:
from scipy.stats import mode  # Option 1: Using scipy

def aggregate_patient_chunks(features_list, labels, chunk_size=10):
    """
    Aggregate features from multiple chunks for each patient.
    
    Args:
        features_list: List of feature dictionaries
        labels: Corresponding labels
        chunk_size: Number of chunks to aggregate
    """
    n_chunks = len(features_list)
    n_groups = n_chunks // chunk_size
    
    aggregated_features = []
    aggregated_labels = []
    
    for i in range(n_groups):
        start_idx = i * chunk_size
        end_idx = start_idx + chunk_size
        chunk_group = features_list[start_idx:end_idx]
        
        # Aggregate MFCC features
        mfcc_means = np.array([chunk['mfcc_mean'] for chunk in chunk_group])
        mfcc_stds = np.array([chunk['mfcc_std'] for chunk in chunk_group])
        
        # Calculate statistics over the chunks
        group_features = {
            # Mean and std of MFCC means across chunks
            'mfcc_mean_mean': np.mean(mfcc_means, axis=0),
            'mfcc_mean_std': np.std(mfcc_means, axis=0),
            # Mean and std of MFCC stds across chunks
            'mfcc_std_mean': np.mean(mfcc_stds, axis=0),
            'mfcc_std_std': np.std(mfcc_stds, axis=0),
            # Other features averaged across chunks
            'spectral_centroid_mean': np.mean([chunk['spectral_centroid_mean'] for chunk in chunk_group]),
            'spectral_bandwidth_mean': np.mean([chunk['spectral_bandwidth_mean'] for chunk in chunk_group]),
            'spectral_rolloff_mean': np.mean([chunk['spectral_rolloff_mean'] for chunk in chunk_group]),
            'zero_crossing_rate_mean': np.mean([chunk['zero_crossing_rate_mean'] for chunk in chunk_group]),
            'pitch_mean': np.mean([chunk['pitch'] for chunk in chunk_group]),
            'energy_mean': np.mean([chunk['energy'] for chunk in chunk_group]),
            'speech_rate_mean': np.mean([chunk['speech_rate'] for chunk in chunk_group])
        }
        
        # Flatten the features into a single vector
        feature_vector = np.concatenate([
            group_features['mfcc_mean_mean'],
            group_features['mfcc_mean_std'],
            group_features['mfcc_std_mean'],
            group_features['mfcc_std_std'],
            [group_features['spectral_centroid_mean']],
            [group_features['spectral_bandwidth_mean']],
            [group_features['spectral_rolloff_mean']],
            [group_features['zero_crossing_rate_mean']],
            [group_features['pitch_mean']],
            [group_features['energy_mean']],
            [group_features['speech_rate_mean']]
        ])
        
        aggregated_features.append(feature_vector)
        # Take the most common label in the chunk group
        aggregated_labels.append(mode(labels[start_idx:end_idx]))
    
    return np.array(aggregated_features), np.array(aggregated_labels)

In [34]:
X_aggregated, y_aggregated = aggregate_patient_chunks(organized_features, labels, chunk_size=5)


In [35]:
y_aggregated

array([[9, 5],
       [9, 5],
       [9, 5],
       [9, 5],
       [9, 5],
       [9, 5],
       [9, 5],
       [9, 5],
       [9, 5]], dtype=int64)

In [ ]:
def train_decision_tree(features, labels):
    """Train and evaluate a decision tree classifier"""
    # Create and train the decision tree
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(features, labels)
    return clf

In [ ]:
# Aggregate features
X_aggregated, y_aggregated = aggregate_patient_chunks(organized_features, labels, chunk_size=10)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_aggregated, y_aggregated, test_size=0.2, random_state=42)

# Train model
clf = train_decision_tree(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Print results
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))